# 대화형 검색 - Conversational Search


# Overview

대화형 검색은 사용자가 자연스럽게 질문하면 인공지능 시스템이 이해하고 관련 정보를 제공하는 방식을 말합니다. 단순히 키워드를 검색하는 것이 아니라, 실제 대화를 통해 정보를 얻습니다. 이는 사용자가 보다 자연스럽게 원하는 정보를 찾을 수 있게 해주며, 키워드를 입력하는 대신 대화를 통해 원하는 내용을 명확히 전달할 수 있습니다. 더불어, 지속적인 대화를 통해 추가적인 정보를 요청하거나 세부 사항을 파악하는 것도 가능합니다.


# 사전 준비

이번 단계를 진행하기 위해서는 [시맨틱 검색 단계](./02.semantic_search.ipynb)를 필수적으로 완료하셔야 합니다. Amazon OpenSearch Service로의 연결은 [시맨틱 검색 단계](./02.semantic_search.ipynb)와 동일하게 수행합니다.


In [2]:
%store -r model_id
%store -r index_name

패키지를 설치하고 import 합니다.


CloudFormation에서 필요한 정보를 가져와 OpenSearch 도메인에 연결합니다.


In [13]:
from opensearchpy import OpenSearch
import textwrap
import json

# OpenSearch 연결 설정
base_url = "https://localhost:9200/_plugins/_ml"
host = 'localhost'
port = 9200
auth = ('admin', 'TestUser2@')  # 초기 설정한 어드민 비밀번호 사용

aos_client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False,
)

연결이 잘 되었는지 확인합니다.


In [ ]:
import requests

# search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

# response = requests.get(
#     "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
# )
# model_info = json.loads(response.text)
# model_id = model_info["hits"]["hits"][0]["_id"]

# index_name = "movie_semantic"

In [4]:
count = aos_client.count(index=index_name)
print(count)

{'count': 1000, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


# 대화형 검색 구현하기


## LangChain을 사용한 대화형 AI 에이전트 구성

LangChain 라이브러리를 사용하여 Anthropic의 Claude 언어 모델을 기반으로 한 대화형 AI 에이전트를 구성합니다. 여기서는 **`ConversationBufferWindowMemory`**를 사용하여 최근 10개의 메시지를 기억하는 대화 기억 장치를 생성합니다.


In [6]:
import os
from dotenv import dotenv_values

env_vars = dotenv_values('.env_api')
os.environ["OPENAI_API_KEY"] = env_vars.get("OPENAI_API_KEY")

In [7]:
from langchain.memory import ConversationBufferWindowMemory

from langchain.chains.question_answering import load_qa_chain
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(
    model_name='gpt-4o-mini-2024-07-18',
    max_tokens=2048,
    temperature=0,
    streaming=True,
)

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=10, return_messages=True)

## OpenSearch의 하이브리드 Retriever 객체 생성

OpenSearch 엔진을 사용하여 하이브리드 검색을 수행하는 OpenSearchHybridSearchRetriever를 정의합니다. 이 검색기는 LangChain의 BaseRetriever 클래스를 상속받고 있으며, 주요 기능은 다음과 같습니다. OpenSearchHybridSearchRetriever는 추후 대화형 검색 체인에서 retriever 역할을 담당하게 됩니다.


In [8]:
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import BaseRetriever
from typing import Any, List
from langchain.schema import Document


class OpenSearchHybridSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    model_id: str
    keyword_weight = 0.3
    semantic_weight = 0.7
    k = 10
    minimum_should_match = 0
    filter = []

    def _reset_search_params(
        self,
    ):

        self.k = 10
        self.minimum_should_match = 0
        self.filter = []
        self.keyword_weight = keyword_weight
        self.semantic_weight = semantic_weight

    def _get_relevant_documents(
        self, query_text: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        query = {
            "size": 10,
            "_source": {"exclude": ["text", "vector_field"]},
            "query": {
                "hybrid": {
                    "queries": [
                        {
                            "multi_match": {
                                "query": query_text,
                                "fields": ["title", "plot", "genre", "main_act", "supp_act"],
                            }
                        },
                        {
                            "neural": {
                                "vector_field": {
                                    "query_text": query_text,
                                    "model_id": model_id,
                                    "k": 30,
                                }
                            }
                        },
                    ]
                }
            },
            "search_pipeline": {
                "description": "Post processor for hybrid search",
                "phase_results_processors": [
                    {
                        "normalization-processor": {
                            "normalization": {"technique": "min_max"},
                            "combination": {
                                "technique": "arithmetic_mean",
                                "parameters": {
                                    "weights": [self.keyword_weight, self.semantic_weight]
                                },
                            },
                        }
                    }
                ],
            },
        }
        res = self.os_client.search(index=index_name, body=query)

        query_result = []

        for hit in res["hits"]["hits"]:
            metadata = {"score": hit["_score"], "id": hit["_id"]}

            content = {
                "제목": hit["_source"]["title"],
                "장르": hit["_source"]["genre"],
                "평점": hit["_source"]["rating"],
                "줄거리": hit["_source"]["plot"],
                "주연": hit["_source"]["main_act"],
                "조연": hit["_source"]["supp_act"],
            }

            doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)
            query_result.append(doc)
        return query_result

## 프롬프트 템플릿 정의


### 영화 추천 기본 프롬프트 정의

이 프롬프트는 목록에서 특정 질문에 대한 답변을 생성하는 프롬프트 템플릿을 정의합니다. 프롬프트 템플릿은 주어진 컨텍스트(영화 목록)와 질문을 입력받아 답변을 생성하는 형식으로 구성되어 있습니다. 답변 생성 시에는 정해진 규칙(예의바른 태도, 목록 형식, 간단한 설명 등)을 따르도록 되어 있습니다.


In [9]:
from langchain import PromptTemplate

prompt_template = """


Human: Here is the list of recommended movies, inside <movies></movies> XML tags.

<movies>
{context}
</movies>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.
    - Answered in list format
    - Always put a short and concise explanation on why you are recommending this movies.

You are a best movie reviewer in Korea. Please answer the movie question based on the information in the context.

Assistant: Here are a movies from the list above.

Question:
{question}

If the answer is not in the context, just say "추천해드릴만한 영화가 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

### Condense 템플릿 정의

이 프롬프트 템플릿은 대화 내역과 후속 질문을 입력받아 독립적인 질문을 생성하는 데 사용됩니다. 템플릿의 구조는 다음과 같습니다:

1. **지시사항(instructions)**: 이 섹션에서는 프롬프트의 목적과 입력 형식에 대한 설명을 제공합니다.
2. **대화 내역(chat_history)**: 여기에는 이전 대화 내용이 포함됩니다. **`{chat_history}`** 부분이 실제 대화 내역으로 대체됩니다.
3. **후속 질문(follow-up-question)**: 이 섹션에는 대화 내역에 대한 후속 질문이 포함됩니다. **`{question}`** 부분이 실제 후속 질문으로 대체됩니다.
4. **독립 질문(standalone question)**: 이 부분에서 모델은 대화 내역과 후속 질문을 요약한 독립적인 질문을 생성해야 합니다.

이 프롬프트 템플릿을 사용하면 LangChain의 **`CONDENSE_QUESTION_PROMPT`**에 대화 내역과 후속 질문을 입력할 수 있습니다. 그러면 모델이 이전 대화 내용과 후속 질문을 종합하여 독립적인 질문을 생성합니다. 이렇게 생성된 질문은 대화 맥락을 유지하면서도 독립적으로 이해할 수 있습니다.


In [10]:
condense_template = """
Generate one standalone question based on the instructions.

<instrunctions>
- You will be given the following conversation between <chat-history> and </chat-history>
- You will be given the following follow up question between <follow-up-question> and </follow-up-question>
- Standalone question should have summary of the previous questions and answers.
</instructions>

<chat-history>
{chat_history}
</chat-history>

<follow-up-question>
{question}
</follow-up-question>

standalone question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

ConversationalRetrievalChain을 통해 대화형 검색을 위한 Chain을 구성합니다. 문서 검색을 위해 앞서 정의한 OpenSearchHybridSearchRetriever를 retriever로 제공하고, 검색 히스토리가 저장되는 memory 객체를 전달합니다.


In [18]:
from langchain.chains import ConversationalRetrievalChain

memory.clear()

hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=aos_client, index_name=index_name, model_id=model_id
)

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=hybrid_retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    # verbose=True,
)

# 대화형 검색 테스트

이제 실제로 대화형 검색을 시도해봅시다. 먼저 첫 번째 질문으로 영화를 추천받습니다.


In [19]:
first_question = "초능력을 가진 영웅들이 지구를 지킨다"
chat_response = conversation_with_retrieval.invoke({"question": first_question})

print(textwrap.fill(chat_response["answer"], 80))

Here are some movies from the list that feature heroes with special abilities or
powers who protect the Earth:  1. **초능력자 (The Superpower)**    - **Genre:**
SF|스릴러|액션|드라마    - **Rating:** 6.69    - **Summary:** The story revolves around
a character named 규남, who is the only person immune to the control of a
superhuman who can manipulate others. As chaos ensues, 규남 must confront this
powerful adversary to protect himself and others.    - **Recommendation
Reason:** This film showcases a battle between a superhuman and an ordinary
person who must rise to the challenge, embodying the theme of protecting
humanity against extraordinary threats.  2. **왓치맨 (Watchmen)**    - **Genre:**
액션|드라마|SF|판타지|스릴러    - **Rating:** 6.8    - **Summary:** Set in a world where
superheroes are outlawed, the story follows '로어셰크' as he investigates the murder
of a former colleague, uncovering a conspiracy that threatens humanity.    -
**Recommendation Reason:** This film features heroes with unique abilities who

In [20]:
second_question = "그 영화에 출현한 배우는 누구입니까?"
chat_response = conversation_with_retrieval.invoke({"question": second_question})

print(textwrap.fill(chat_response["answer"], 80))

Here are the actors from the recommended movies **초능력자**, **왓치맨**, and **수어사이드
스쿼드**:  1. **초능력자**    - **주연**: 강동원, 고수, 정은채    - **조연**: 윤다경, 최덕문, 아부다드, 에네스
카야, 양경모    - **추천 이유**: 이 영화는 초능력을 가진 인물들 간의 긴장감 넘치는 대결을 그려내며, 강동원과 고수의 뛰어난 연기가
돋보입니다.  2. **왓치맨**    - **주연**: 잭키 얼 헤일리, 제프리 딘 모건, 빌리 크루덥, 말린 애커맨, 칼라 구기노, 패트릭
윌슨, 매튜 구드    - **조연**: 스티븐 맥허티, 맷 플레워, 로라 메넬, 대니 우드번, 나이얼 매터, 크리스 고디어, 캐리 겐젤, 샐리
사피오티, 브렛 스티멜리, 제이 브라조, 댄 페인    - **추천 이유**: 이 영화는 복잡한 스토리와 깊이 있는 캐릭터들로 구성되어 있으며,
배우들의 뛰어난 연기가 관객을 사로잡습니다.  3. **수어사이드 스쿼드**    - **주연**: 윌 스미스, 자레드 레토, 마고 로비, 카라
델레바인, 제이 코트니    - **조연**: 조엘 킨나만, 비올라 데이비스, 아데웰 아킨누오예 아바제, 제이 헤르난데즈, 아담 비치, 카렌
후쿠하라, 커먼, 코리나 칼데론, 짐 파랙, 이크 바린홀츠, 에즈라 밀러, 알렉스 메라즈, 케빈 밴스, 밤바디언 밤바, 테드 휘톨, 데이빗
하버, 로빈 앳킨 다운즈, 빌리 오티스, 제임스 맥고완, 스콧 이스트우드    - **추천 이유**: 이 영화는 다양한 캐릭터들이 모여 팀을
이루는 독특한 설정과 함께, 스타 배우들의 화려한 연기가 매력적입니다.  이 영화들은 각기 다른 매력을 지니고 있으며, 출연 배우들의 뛰어난
연기가 영화의 재미를 더해줍니다.


In [21]:
third_question = "비슷한 장르의 다른 영화는?"
chat_response = conversation_with_retrieval.invoke({third_question})

print(textwrap.fill(chat_response["answer"], 80))

Here are some movies from the list above that feature heroes with special
abilities or powers, along with their notable actors:  1. **수어사이드 스쿼드**    -
**Notable Actors**: 윌 스미스, 자레드 레토, 마고 로비    - **Explanation**: This film
showcases a team of supervillains who are forced to work together on a dangerous
mission, highlighting their unique abilities.  2. **어벤져스**    - **Notable
Actors**: 로버트 다우니 주니어, 스칼렛 요한슨, 크리스 헴스워스    - **Explanation**: A classic
superhero ensemble film where various heroes with extraordinary powers unite to
save the world from a formidable threat.  3. **앤트맨과 와스프**    - **Notable
Actors**: 폴 러드, 에반젤린 릴리, 마이클 더글라스    - **Explanation**: This film features
heroes who can shrink in size while gaining superhuman strength, adding a unique
twist to the superhero genre.  4. **킥 애스: 영웅의 탄생**    - **Notable Actors**: 애런
존슨, 클로이 모레츠, 니콜라스 케이지    - **Explanation**: A story about an ordinary teenager
who decides to become a superhero, showcasing the journey of self-discovery and
h

In [27]:
print(memory.chat_memory)

Human: 초능력을 가진 영웅들이 지구를 지킨다
AI: Here are some movies from the list that feature heroes with special abilities or powers who protect the Earth:

1. **초능력자 (The Superpower)**
   - **Genre:** SF|스릴러|액션|드라마
   - **Rating:** 6.69
   - **Summary:** The story revolves around a character named 규남, who is the only person immune to the control of a superhuman who can manipulate others. As chaos ensues, 규남 must confront this powerful adversary to protect himself and others.
   - **Recommendation Reason:** This film showcases a battle between a superhuman and an ordinary person who must rise to the challenge, embodying the theme of protecting humanity against extraordinary threats.

2. **왓치맨 (Watchmen)**
   - **Genre:** 액션|드라마|SF|판타지|스릴러
   - **Rating:** 6.8
   - **Summary:** Set in a world where superheroes are outlawed, the story follows '로어셰크' as he investigates the murder of a former colleague, uncovering a conspiracy that threatens humanity.
   - **Recommendation Reason:** This film features 

In [17]:
%store model_id
%store index_name

Stored 'model_id' (str)
Stored 'index_name' (str)
